# GSC to CSV




#### Search Console Data

##### Getting a Google API Credential.

1) Sign up for a new project in the Google APIs console at https://code.google.com/apis/console.

2) Choose "Credentials" on the left-hand menu.  

3) Choose "Create Credentials"

4) Generate an API key for your application.

5) Copy your client ID, client secret, and redirect URL into the next cell

get data by week if not recursively go through the data daily



In [ ]:
#@title Configure your access
import httplib2

from apiclient import errors
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow

# Copy your credentials from the console
CLIENT_ID = '87393635730-0o6ctsjmronvvn0qfh3j9jeomiccpqbj.apps.googleusercontent.com' #@param {type:"string"}
CLIENT_SECRET = 'XQu8YEO3C5b1jV4wo18UDFsH' #@param {type:"string"}

# Check https://developers.google.com/webmaster-tools/search-console-api-original/v3/ for all available scopes
OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'

# Redirect URI for installed apps
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

# Run through the OAuth flow and retrieve credentials
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print('Go to the following link in your browser: ' + authorize_url)
code = input('Enter verification code: ').strip()
credentials = flow.step2_exchange(code)

__init__() takes at most 4 positional arguments (5 given)


Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=87393635730-0o6ctsjmronvvn0qfh3j9jeomiccpqbj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code
Enter verification code: 4/1AY0e-g50X8G7CvBhF-95aYSm8G2ikixK_3pgNSQw0xmyrtvEFEG2JLgSuWE


In [ ]:
#@title Choose your projects
#@markdown Generate your selector

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()
http = credentials.authorize(http)

webmasters_service = build('webmasters', 'v3', http=http)

# Retrieve list of properties in account
site_list = webmasters_service.sites().list().execute()

# Filter for verified websites
verified_sites_urls = [s['siteUrl'] for s in site_list['siteEntry']
                       if s['permissionLevel'] != 'siteUnverifiedUser']
                    #      and s['siteUrl'][:4] == 'http']

# Print the URLs of all websites you are verified for.
projects = []
for site_url in verified_sites_urls:
  print(site_url)
  projects.append(('{}'.format(site_url), site_url))

# print(domain_url)
# Create Selector
import ipywidgets as widgets
output = widgets.Output()

dropdown_purpose = widgets.Dropdown(options = projects, description="Site: ")

def dropdown_project_eventhandler(change):
    output.clear_output()    
    with output:
        display(projects)

dropdown_purpose.observe(dropdown_project_eventhandler, names='value')

display(dropdown_purpose)

https://lolsurprise.mgae.com/
https://babyborn.mgae.com/
sc-domain:contentaudience.com
sc-domain:liston.io
sc-domain:servedontsell.co
https://extricareusa.com/
https://lalaloopsy.mgae.com/
https://www.mgae.com/
https://whatsinmypursesurprise.mgae.com/
https://anchoradvisors.com/
sc-domain:philipmorganconsulting.com
http://polycor.com/
https://indiyoung.com/
https://smithhonig.com/
https://poopsie.mgae.com/
sc-domain:polycor.com
https://www.lolsurprise.com/
http://www.nfcamenitymanagement.com/
sc-domain:anchoradvisors.com
sc-domain:designasylumblog.com
sc-domain:plumflowersoftware.com
https://www.2percentcommission.com/
https://inboundfound.com/
https://sbstandard.com/
https://dojobattle.mgae.com/
sc-domain:littletikes.com
https://shreddinsharks.mgae.com/
https://numnoms.mgae.com/
http://themoriuchigroup.com/
https://www.servedontsell.co/
https://hardscapes.polycor.com/
sc-domain:servedontsell.com
https://virorides.mgae.com/
http://philipmorganconsulting.com/
https://contentaudience.com

Dropdown(description='Site: ', options=(('https://lolsurprise.mgae.com/', 'https://lolsurprise.mgae.com/'), ('…

In [ ]:
#@title Get your data
#@markdown Note that if you specify a 30-day period, the script will split the data in two and compare the last 15 days with the first 15.
import pandas as pd
from dateutil import rrule
from datetime import datetime, timedelta, date

url = ""
if(len(dropdown_purpose.value)==0):
  print("no Site detected")
else:
  url = dropdown_purpose.value

# init
DF = pd.DataFrame(columns=['page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'date' ])
## start of get data
def get_data(startDate, endDate, base= rrule.DAILY):
  '''
  startDate: date string with the following format %Y-%m-%d
  '''
  DF = pd.DataFrame(columns=['page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'date' ])
  request = {
    'startDate': startDate,    
    'endDate': '2100-12-29',    
    'dimensions': ["page", "query"],
    'searchType': "web",      
    'responseAggregationType': 'page',
    'rowLimit': 25000
  }

  for dt in rrule.rrule(base, dtstart=datetime.strptime(startDate, '%Y-%m-%d'), until=datetime.strptime(endDate, '%Y-%m-%d'))[1:]:
    request['endDate'] = dt.strftime('%Y-%m-%d')
    response =  webmasters_service.searchanalytics().query(siteUrl=url, body=request).execute()
    try:
      dfGSC = pd.DataFrame.from_dict(response['rows'], orient='columns')
      # if rows more than 25000 then go daily
      if dfGSC.shape[0] == 25000:
        #daily for one week
        week_start = request['startDate']
        for dt in rrule.rrule(rrule.DAILY, dtstart=datetime.strptime(request['startDate'], '%Y-%m-%d'), until=datetime.strptime(request['endDate'], '%Y-%m-%d'))[1:]:
          
          request['endDate'] = dt.strftime('%Y-%m-%d')
          response =  webmasters_service.searchanalytics().query(siteUrl=url, body=request).execute()
          try:
            dfGSC = pd.DataFrame.from_dict(response['rows'], orient='columns')
            dfGSC[["page", "query"]] = pd.DataFrame( dfGSC["keys"].values.tolist() )
            dfGSC =  dfGSC.drop(columns=['keys'])
            dfGSC['date'] = week_start
            dfGSC = dfGSC[['page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'date' ]]
            #print(dfGSC.keys()) 
            DF = pd.concat([DF, dfGSC], axis=0)
            request['startDate'] = request['endDate']
          except Exception as e:
            print('No Data!')
            break
        continue

      dfGSC[["page", "query"]] = pd.DataFrame( dfGSC["keys"].values.tolist() )
      dfGSC =  dfGSC.drop(columns=['keys'])
      dfGSC['date'] = request['startDate']
      dfGSC = dfGSC[['page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'date' ]]
      #print(dfGSC.keys()) 
      DF = pd.concat([DF, dfGSC], axis=0)
      
    except Exception as e:
      print('No Data!')
      break
    request['startDate'] = request['endDate']
  return DF

total_data = get_data('2021-01-01', '2021-06-02')
total_data.to_csv('gsc-api-daily-www-polycor.csv', index=False)